In [8]:
import src.graphParameters as gP
from src.Graph import Graph
import networkx as nx
from test_.Performancetest import GraphPerformanceTester

In [9]:

def generate_gnp_graph(n, p, netx=False):
    newG = nx.gnp_random_graph(n, p)
    if netx:
        return newG
    return translate_Graph(newG)

def translate_Graph(newG):
    nodes = nx.nodes(newG)
    G = Graph()
    for node in nodes:
        G.add_node(node)
    edges = nx.edges(newG)
    for edge in edges:
        G.add_edge(edge[0], edge[1])
    return G

In [10]:
def get_stats(G: Graph):
    stats = {}
    stats["density"] = gP.density(G)
    stats["max_degree"] = gP.max_degree(G)
    deg, _ = gP.degeneracy(G)
    stats["degeneracy"] = deg
    stats["global_clustering"] = gP.global_clustering_coefficient(G)
    stats["h_index"] = gP.h_index(G)
    return stats


In [11]:
import os
import pandas as pd

graph_files = []
for file in os.listdir("networks/power_networks"):
    graph_files.append(file)
rand_names = ["rand_"+str(i) for i in range(len(graph_files))]

columns = ["graph_name","nodes","edges", "density", "max_degree", "degeneracy", "global_clustering", "h_index"]
df = pd.DataFrame(columns=columns)
for file,rand_name in zip(graph_files,rand_names):
    path = "networks/power_networks/"+file
    G = Graph(path)
    stats = get_stats(G)
    row = [file,G.n, G.m, stats["density"], stats["max_degree"], stats["degeneracy"], stats["global_clustering"], stats["h_index"]]
    df.loc[len(df)] = row
    rand_G = generate_gnp_graph(G.n, stats["density"])
    stats = get_stats(rand_G)
    row = [rand_name, rand_G.n, rand_G.m, stats["density"], stats["max_degree"], stats["degeneracy"], stats["global_clustering"], stats["h_index"]]
    df.loc[len(df)] = row


In [12]:
df

,graph_name,nodes,edges,density,max_degree,degeneracy,global_clustering,h_index
0,power-1138-bus.mtx,1138,1458,0.002254,17,4,0.093068,9
1,rand_0,1138,1388,0.002145,8,2,0.002639,7
2,power-494-bus.mtx,494,586,0.004812,9,2,0.048424,7
3,rand_1,494,603,0.004952,7,2,0.000000,6
4,power-662-bus.mtx,662,906,0.004141,9,4,0.076856,8
5,rand_2,662,925,0.004228,9,2,0.003577,7
6,power-685-bus.mtx,685,1282,0.005472,12,5,0.214471,9
7,rand_3,685,1277,0.005451,11,3,0.005746,8
8,power-US-Grid.mtx,4941,6594,0.000540,19,5,0.103153,12
9,rand_4,4941,6713,0.000550,11,2,0.000325,8


In [13]:
import time
def get_times(G):
    all_local_cluster = lambda x : [gP.local_clustering_coefficient(x, str(i)) for i in x.node_ids_internal_ids.keys()]
    funcs = [gP.degeneracy, gP.k_core_decomposition, gP.h_index, gP.global_clustering_coefficient, all_local_cluster]
    for f in funcs:
        tick = time.time()
        f(G)
        tack = time.time()
        print(tack-tick, f.__name__)

N = 100
for i in range(5):
    rand_G = generate_gnp_graph(N,0.03)
    N = N*5
    print(N)
    get_times(rand_G)


500
0.0068895816802978516 degeneracy
0.006506204605102539 k_core_decomposition
0.00010609626770019531 h_index
0.0005276203155517578 global_clustering_coefficient
0.00044608116149902344 <lambda>
2500
0.05286908149719238 degeneracy
0.06035304069519043 k_core_decomposition
0.00014662742614746094 h_index
0.005093574523925781 global_clustering_coefficient
0.00610661506652832 <lambda>
12500
1.4120066165924072 degeneracy
1.6366925239562988 k_core_decomposition
0.0015230178833007812 h_index
0.7501895427703857 global_clustering_coefficient
0.7534656524658203 <lambda>
62500
47.067039251327515 degeneracy
48.99715065956116 k_core_decomposition
0.007425785064697266 h_index
72.49999737739563 global_clustering_coefficient
72.88419270515442 <lambda>


KeyboardInterrupt: 